# Installing required Python modules

In [ ]:
# After executing the following commands you will need to restart the Python kernal (from the Kernel menu).
%pip install ipyleaflet
%pip install gtfs-realtime-bindings

# Visualizing GeoSpacial Data

In this week's exercises we will explore a Python module called leaflet that allows us to visualize GeoSpacial data, i.e. data that has a longitude and latitude associated with it.

We will start by revisiting the static GTFS public transport data set that we explored in week 9 and will  extend it to fetching and visualizing real-time vehicle locations.

In [ ]:
# Let's start by using leaflet to create an interactive map
import ipyleaflet
map = ipyleaflet.Map()
map

# Zoom out until you can see some land and then navigate to Australia

In [ ]:
# Start as you did in week 9 by determining the exact longitute and latitude of the property where you live.
my_longitude = # add your values here
my_latitude = # add your values here

In [ ]:
# Now let's change the centre of the map, change the zoom level and change the map width and height
map = ipyleaflet.Map(center=(my_longitude,my_latitude), zoom=15)
map.layout.height="700px"
map.layout.width="1000px"
map

In [ ]:
# Add a marker to the map to show the location of your home

home = ipyleaflet.Marker(location=(my_longitude,my_latitude), draggable=False, icon=ipyleaflet.AwesomeIcon(name="home", marker_color='blue'), title="Home")
map.add_layer(home)

# Stops near me ...

The following is a repeat of what we did in the week 9 partical exercises

In [ ]:
import pandas
stops = pandas.read_csv('stops.txt', index_col = 0)

In [ ]:
import math

# https://en.wikipedia.org/wiki/Haversine_formula
def haversine_distance(lon1, lat1, lon2, lat2):
      # convert decimal degrees to radians 
      lon1 = math.radians(lon1)
      lat1 = math.radians(lat1)
      lon2 = math.radians(lon2)
      lat2 = math.radians(lat2)
        
      # haversine formula 
      dlon = lon2 - lon1 
      dlat = lat2 - lat1 
      a =  math.sin(dlat/2)**2 +  math.cos(lat1) * math.cos(lat2) *  math.sin(dlon/2)**2
      c = 2 * math.asin( math.sqrt(a)) 
      r = 6371 # Radius of earth in kilometers.
      return c * r

In [ ]:
def near(stop_row, lon, lat) :
    return haversine_distance(lon, lat, stop_row.stop_lat, stop_row.stop_lon)

stops['dist_from_home'] = stops.apply(near, lon=my_longitude, lat=my_latitude, axis=1)

nearby_stops = stops.sort_values('dist_from_home')
nearby_stops[:10]

# Visualize stops near me ...

In [ ]:
# We now want to visualize the 10 stops closest to our home.
# To do that, we will need to write a for loop that iterates through the rows in our Pandas dataframe:

for index,stop in nearby_stops[:10].iterrows() :
    print(index, stop[1], stop[2], stop[3], stop[4])

In [ ]:
# change the above loop so that it creates markers for each stop.
# change the marker location so that it is based on the longitude and latitude of each stop
# change the icon to show a "bus" rather than a home (https://fontawesome.com/v4/icons/)
# change the marker colour to green
# change the mouse over title to be the stop_id followed by the stop name

for index,stop in nearby_stops[:10].iterrows() :
    # add your code here

# Select a bus stop

In [ ]:
# When visualizing the stops, if you put your mouse over an icon it will show the title which includes the stop_id
# Select one of those stop_ids to explore further
our_stop_id = ??? # make sure it is expressed as a 'string' rather than as an integer (as some stop_ids are not numeric)

# Find buses departing from my stop soon ...

The following is a repeat of what we did in the week 9 partical exercises

In [ ]:
stop_times = pandas.read_csv('stop_times.txt', dtype={'stop_id':'str'})
services = pandas.read_csv('calendar.txt', index_col = 0, parse_dates=['start_date','end_date'])

In [ ]:
import pytz
timezone = pytz.timezone('Australia/Brisbane')
today = pandas.Timestamp.now(tz=timezone).tz_localize(None)

## Make sure your update the day in the following query to reflect the current day of the week ...

In [ ]:
todays_services = services[(services.thursday == 1) & (services.start_date <= today) & (today <= services.end_date)].index

In [ ]:
trips = pandas.read_csv('trips.txt', index_col = 2)

In [ ]:
todays_trips = trips[trips.service_id.isin(todays_services)].index

In [ ]:
time_now = today.strftime('%H:%M:%S')

arriving_soon = stop_times[(stop_times.stop_id==our_stop_id) & (stop_times.trip_id.isin(todays_trips)) & (time_now <= stop_times.arrival_time)  ]

In [ ]:
stops_with_trips = arriving_soon.join(trips, on='trip_id')

In [ ]:
routes = pandas.read_csv('routes.txt', index_col = 0)

In [ ]:
full = arriving_soon.join(trips, on='trip_id').join(routes, on='route_id')

In [ ]:
show = full[['trip_id','arrival_time', 'route_short_name', 'route_long_name', 'trip_headsign']]
show

## Select a trip

In [ ]:
# Select one of these trips to explore further ...
our_trip_id = ???

In [ ]:
my_stops = stop_times[stop_times.trip_id == our_trip_id]
full_stop_data = my_stops.join(stops, on='stop_id')[['arrival_time', 'stop_name', 'stop_lat', 'stop_lon']]
full_stop_data

# Visualize the stops on the trip ...

In [ ]:
# Add markers for each of these stops to your map.
# Use a different colour and icon from what you used previously for stops near you (https://fontawesome.com/v4/icons/)
# The mouse over title should be the arrival time followed  by the stop name

# Add your code here

# Get Realtime vehicle position feed ...

Next we will fetch some real-time GTFS data (updated every 30 seconds) to get the current location of buses around Brisbane

In [ ]:
# Because these bus locations will get updated, we want to be able to easily remove the old bus icons and add new icons at the updated locations
# To do so, rather than adding these markers directly to the map, we will instead create a Layer group and add these real-time bus locations to that layer
bus_markers = ipyleaflet.LayerGroup()
# We then add this layer group to the map
map.add_layer(bus_markers)

In [ ]:
# First we need to make a HTTPS request to fetch the latest data from the translink website 
import requests
response = requests.get('https://gtfsrt.api.translink.com.au/api/realtime/SEQ/VehiclePositions')

In [ ]:
# If you were to  view the response.content, you would see that the information is encoded in a binary format.
# To decode that binary format, we need to import a GTFS module.

from google.transit import  gtfs_realtime_pb2
feed = gtfs_realtime_pb2.FeedMessage()

In [ ]:
# We can now parse the response using this GTFS feed object
feed.ParseFromString(response.content)

In [ ]:
# The feed now provides us with a list of entities.
# Let's start by viewing the first entity in the list ...

update = feed.entity[0]
print(update)

In [ ]:
# We can see that the update contains and id, and a vehicle. The vehicle then contains a trip and a position, and the trip contains a trip_id etc.
# So, we can, for example access the trip_id as follows
print(update.vehicle.trip.trip_id)

In [ ]:
# write a similar expression to access the longitude and latitude of the vehicle

# add your code here

In [ ]:
# Finally we will iterate through all the entities and add markers to our bus marker layer for every bus.
# Change the location based on the longitude and latitude of the vehicle
# Change icon and colour (https://fontawesome.com/v4/icons/)
# Change the mouse over title to be the trip_id
bus_markers.clear_layers()
for entity in feed.entity :
    # add your code here ...
    bus_markers.add_layer(bus_marker)    